In [74]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    model="gpt-4o-mini",
    tiktoken_model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", ("You are a helpful AI talking to a human. Only reply in Korean.")),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

In [75]:
def load_memory():
    return memory.load_memory_variables({})["history"]


chain = prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question, "history": load_memory()})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )


In [76]:
invoke_chain("내 이름은 형진이야.")

안녕하세요, 형진님! 만나서 반가워요. 어떻게 도와드릴까요?

In [77]:
invoke_chain("나는 지금 일본의 고베에 살고 있어.")

고베에 살고 계시군요! 고베는 아름다운 도시죠. 그곳에서 어떤 활동을 즐기시나요?The human introduces himself as Hyungjin. The AI greets him warmly and asks how it can assist him.

In [78]:
invoke_chain("내 이름이 뭐야?")

당신의 이름은 형진이죠!

In [79]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human introduces himself as Hyungjin. The AI greets him warmly and asks how it can assist him.'),
  HumanMessage(content='나는 지금 일본의 고베에 살고 있어.'),
  AIMessage(content='고베에 살고 계시군요! 고베는 아름다운 도시죠. 그곳에서 어떤 활동을 즐기시나요?'),
  HumanMessage(content='내 이름이 뭐야?'),
  AIMessage(content='당신의 이름은 형진이죠!')]}